In [1]:
import os
import numpy as np
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input, Model, layers, optimizers, losses, metrics, datasets

root_URL = 'C:\\Users\\lenovo\\Desktop\\'

In [2]:
# 加载数据
pickle_file = root_URL + 'svhn_dataset_112x56.pkl'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_images = save['train_images']
    train_labels = save['train_labels']
    test_images = save['test_images']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Train Shape:', train_images.shape, train_labels.shape)
    print('Test Shape:', test_images.shape, test_labels.shape)

Train Shape: (33401, 56, 112) (33401, 6)
Test Shape: (13068, 56, 112) (13068, 6)


In [3]:
# 预处理函数
def preprocess(x, y):
    x = tf.expand_dims(x, axis=-1) # 在axis=3扩展一维
    y = [y[:,0],y[:,1],y[:,2],y[:,3],y[:,4],y[:,5]]
    return x, y

In [4]:
train_images, train_labels = preprocess(train_images,train_labels)
test_images, test_labels = preprocess(test_images,test_labels)
print(train_images.shape,test_images.shape)

(33401, 56, 112, 1) (13068, 56, 112, 1)


In [5]:
print(min(train_labels[:][0]),max(train_labels[:][0]))

1 5


In [6]:

inputs = Input(shape = (56,112,1))

model = layers.BatchNormalization()(inputs)
model = layers.Conv2D(64, (7,7), padding='same', activation='relu')(model)
model = layers.MaxPool2D(pool_size=2)(model)

model = layers.BatchNormalization()(model)
model = layers.Conv2D(128, (5,5), padding='valid', activation='relu')(model)
model = layers.MaxPool2D(pool_size=2)(model)

model = layers.BatchNormalization()(model)
model = layers.Conv2D(256, (3,3), padding='valid', activation='relu')(model)
model = layers.MaxPool2D(pool_size=2)(model)
model = layers.Dropout(0.5)(model)

model = layers.Flatten()(model)
# model = layers.Dense(1024, activation='relu')(model)
model = layers.Dense(512, activation='relu')(model)

output_0 = layers.Dense(6,activation='softmax')(model)
output_1 = layers.Dense(11,activation='softmax')(model)
output_2 = layers.Dense(11,activation='softmax')(model)
output_3 = layers.Dense(11,activation='softmax')(model)
output_4 = layers.Dense(11,activation='softmax')(model)
output_5 = layers.Dense(11,activation='softmax')(model)

outputs = [output_0, output_1, output_2, output_3, output_4, output_5]

In [7]:
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 56, 112, 1)] 0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 56, 112, 1)   4           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 56, 112, 64)  3200        batch_normalization[0][0]        
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 28, 56, 64)   0           conv2d[0][0]                     
______________________________________________________________________________________________

In [8]:
# 设置回调功能
filepath = 'my_model_whole.h5' # 保存模型地址
saved_model = tf.keras.callbacks.ModelCheckpoint(filepath, verbose = 1) # 回调保存模型功能
tensorboard = tf.keras.callbacks.TensorBoard() # 回调可视化数据功能

In [ ]:
# 执行训练与验证
history = model.fit(train_images, train_labels, 
                    batch_size = 4, epochs = 100,
                    validation_split = 0.2,
                    callbacks = [saved_model, tensorboard])

In [ ]:
model = tf.keras.models.load_model('my_model_whole.h5')
correct = 0
test_len = len(test_images)
for i in range(test_len):
    x = tf.expand_dims(test_images[i], axis=0)
    y0 = tf.expand_dims(test_labels[0][i], axis=0)
    y1 = tf.expand_dims(test_labels[1][i], axis=0)
    y2 = tf.expand_dims(test_labels[2][i], axis=0)
    y3 = tf.expand_dims(test_labels[3][i], axis=0)
    y4 = tf.expand_dims(test_labels[4][i], axis=0)
    y5 = tf.expand_dims(test_labels[5][i], axis=0)
    y = [y0, y1, y2, y3, y4, y5]

#     print(y)
    val = model.evaluate(x, y,verbose=0)
#     print(val)
    acc = tf.reduce_prod(val[7:])
    
    if acc == 1:
        correct += 1

accuracy = correct / test_len
print(accuracy)